In [20]:
# Full Exercise: Running Circuits on Real Hardware with Qiskit Runtime

# --- Imports ---
from qiskit import transpile
from qiskit.visualization import plot_gate_map, plot_histogram
from qiskit_aer import AerSimulator
from qiskit_ibm_runtime import Sampler
from qiskit_ibm_runtime import QiskitRuntimeService

# --- 1. Set up your IBM Quantum Runtime Service ---
token = "def5481c2be968c7b590e4283a82a78730c8921c2fecbd8d04f3c0156e92c1af58109c5ff02eb8eacad50d2f360691c59145c94411d3073daf0610a50ab10f44"
from qiskit_ibm_runtime import QiskitRuntimeService
 
QiskitRuntimeService.save_account(
  token=token,
  channel="ibm_quantum", # `channel` distinguishes between different account types
  overwrite=True  # Set to True to overwrite existing account if needed
)


# --- 2. List available backends and select least busy real device ---


In [ ]:
# Full Exercise: Running Circuits on Real Hardware with Qiskit Runtime

# --- Imports ---
from qiskit import QuantumCircuit, transpile
from qiskit.visualization import plot_gate_map, plot_histogram
from qiskit_aer import AerSimulator
from qiskit_ibm_runtime import Sampler
from qiskit_ibm_runtime import QiskitRuntimeService

# =============================================================================
# 1. Initialize Service from Saved Account
# =============================================================================

# Load the service using your saved account (ibm_quantum channel)
service = QiskitRuntimeService(channel="ibm_quantum")
print(" Service loaded from saved account")

# =============================================================================
# 2. Explore Available Backends
# =============================================================================

print("\n Available backends:")
backends = service.backends()
for i, backend in enumerate(backends):
    if i < 10:  # Show first 10 backends
        status = backend.status()
        queue_length = getattr(status, 'pending_jobs', 'N/A')
        operational = getattr(status, 'operational', 'N/A')
        print(f"  - {backend.name}: operational={operational}, queue={queue_length}")

# Select the least busy operational backend
quantum_backend = service.least_busy(operational=True, simulator=False)
print(f"\n Selected backend: {quantum_backend.name}")

# Display backend status
backend_status = quantum_backend.status()
print(f"Backend status: {backend_status}")

# Plot the gate map of the selected backend
print(f"\n Plotting gate map for {quantum_backend.name}...")
try:
    plot_gate_map(quantum_backend)
    print("Gate map plotted successfully")
except Exception as e:
    print(f"Could not plot gate map: {e}")

# =============================================================================
# 3. Create Bell Circuit
# =============================================================================

print("\n Creating Bell circuit...")
bell = QuantumCircuit(2, 2)
bell.h(0)          # Hadamard gate on qubit 0
bell.cx(0, 1)      # CNOT gate between qubits 0 and 1
bell.measure_all() # Measure all qubits

print("Bell circuit created:")
print(bell.draw())

# Visualize the circuit
try:
    bell.draw(output='mpl')
    print("Circuit diagram plotted")
except:
    print("Circuit diagram plotting not available in this environment")

# =============================================================================
# 4. Transpile Circuit with Optimization Level 0
# =============================================================================

print(f"\n🔧 Transpiling circuit for {quantum_backend.name} with optimization level 0...")

bell_transpiled_0 = transpile(
    bell,
    backend=quantum_backend,
    optimization_level=0
)

print(f"Transpiled circuit (Level 0):")
print(f"  Depth: {bell_transpiled_0.depth()}")
print(f"  Operations count: {bell_transpiled_0.count_ops()}")

try:
    bell_transpiled_0.draw('mpl', idle_wires=False)
    print("Transpiled circuit (Level 0) plotted")
except:
    print("Transpiled circuit plotting not available")

# =============================================================================
# 5. Transpile Circuit with Optimization Level 2
# =============================================================================

print(f"\n🔧 Transpiling circuit for {quantum_backend.name} with optimization level 2...")

bell_transpiled_2 = transpile(
    bell,
    backend=quantum_backend,
    optimization_level=2
)

print(f"Transpiled circuit (Level 2):")
print(f"  Depth: {bell_transpiled_2.depth()}")
print(f"  Operations count: {bell_transpiled_2.count_ops()}")

try:
    bell_transpiled_2.draw('mpl', idle_wires=False)
    print("Transpiled circuit (Level 2) plotted")
except:
    print("Transpiled circuit plotting not available")

# Compare optimization levels
print(f"\n Optimization Comparison:")
print(f"  Level 0 - Depth: {bell_transpiled_0.depth()}, Ops: {sum(bell_transpiled_0.count_ops().values())}")
print(f"  Level 2 - Depth: {bell_transpiled_2.depth()}, Ops: {sum(bell_transpiled_2.count_ops().values())}")

# =============================================================================
# 6. Run on Aer Simulator
# =============================================================================

print(f"\n  Running on Aer simulator...")

# Set up Aer simulator
aer_sim = AerSimulator()
sampler = Sampler(mode=aer_sim)
shots = 500

# Run the simulation
quantum_job_aer = sampler.run([bell_transpiled_2], shots=shots)
quantum_result_aer = quantum_job_aer.result()
quantum_dist_aer = quantum_result_aer[0].data.c.get_counts()

print(f"Aer Simulator Results ({shots} shots): {quantum_dist_aer}")

# =============================================================================
# 7. Run Job on Physical Hardware (No Error Mitigation)
# =============================================================================

print(f"\n🔮 Submitting job to real quantum hardware: {quantum_backend.name}")

# Create sampler for real hardware using context manager
with Sampler(backend=quantum_backend) as sampler:
    # Disable error mitigation as requested
    sampler.options.dynamical_decoupling.enable = False
    sampler.options.twirling.enable_measure = False

    print("Error mitigation disabled (dynamical_decoupling=False, twirling=False)")

    # Submit the job
    quantum_job = sampler.run([bell_transpiled_2], shots=shots)

print(f" Job submitted successfully!")
print(f"Job ID: {quantum_job.job_id()}")
print(f"Initial Status: {quantum_job.status()}")
print(" Job is now in the queue...")

# =============================================================================
# 8. Run Job on Physical Hardware (WITH Error Mitigation)
# =============================================================================

print(f"\n  Submitting job WITH error mitigation...")

# Create new sampler for error mitigation
# Create new sampler for error mitigation using context manager
with Sampler(backend=quantum_backend) as sampler_mitigated:
    # Enable error mitigation
    sampler_mitigated.options.dynamical_decoupling.enable = True
    sampler_mitigated.options.twirling.enable_measure = True

    print("Error mitigation enabled (dynamical_decoupling=True, twirling=True)")

    # Submit the job with error mitigation
    quantum_job_mitigated = sampler_mitigated.run([bell_transpiled_2], shots=shots)
print(f" Job with error mitigation submitted!")
print(f"Mitigated Job ID: {quantum_job_mitigated.job_id()}")
print(f"Initial Status: {quantum_job_mitigated.status()}")

# =============================================================================
# 9. Job Monitoring Functions
# =============================================================================

def check_job_status(job, job_name="Job"):
    """Check and display detailed job status"""
    status = job.status()
    print(f"\n {job_name} Status Report:")
    print(f"  Status: {status}")
    print(f"  Job ID: {job.job_id()}")
    
    # Check for queue information
    if hasattr(status, 'queue_position') and status.queue_position is not None:
        print(f"  Queue position: {status.queue_position}")
    
    if hasattr(status, 'estimated_start_time') and status.estimated_start_time:
        print(f"  Estimated start: {status.estimated_start_time}")
    
    if hasattr(status, 'estimated_completion_time') and status.estimated_completion_time:
        print(f"  Estimated completion: {status.estimated_completion_time}")
    
    return status.name == 'DONE'

def get_job_results(job, job_name="Job"):
    """Safely retrieve job results when complete"""
    try:
        if check_job_status(job, job_name):
            print(f"🎉 {job_name} completed! Retrieving results...")
            result = job.result()
            counts = result[0].data.c.get_counts()
            print(f" {job_name} Results: {counts}")
            return counts
        else:
            print(f" {job_name} not yet complete.")
            return None
    except Exception as e:
        print(f" Error retrieving {job_name} results: {e}")
        return None

# =============================================================================
# 10. Results Comparison (Run after jobs complete)
# =============================================================================

print(f"\n" + "="*70)
print(" RESULTS ANALYSIS - Run these commands when jobs complete:")
print("="*70)

print("""
# Check status of both hardware jobs
check_job_status(quantum_job, "Hardware Job (No Mitigation)")
check_job_status(quantum_job_mitigated, "Hardware Job (With Mitigation)")

# Retrieve results when jobs are done
quantum_dist = get_job_results(quantum_job, "Hardware Job (No Mitigation)")
quantum_dist_mitigated = get_job_results(quantum_job_mitigated, "Hardware Job (With Mitigation)")

# Compare all three results when both hardware jobs complete
if quantum_dist is not None and quantum_dist_mitigated is not None:
    print("\\n Comparing all results:")
    print(f"Aer Simulator:           {quantum_dist_aer}")
    print(f"Real Device (No Mitig.): {quantum_dist}")
    print(f"Real Device (Mitigated): {quantum_dist_mitigated}")
    
    # Plot comparison histogram
    plot_histogram([quantum_dist_aer, quantum_dist, quantum_dist_mitigated], 
                   legend=['Aer Simulator', 'Real Device', 'Real Device - Error Mitigation'],
                   title='Bell State Results Comparison')
""")

# =============================================================================
# 11. Monitoring Instructions
# =============================================================================

print(f"\n NEXT STEPS:")
print(f"1. Your jobs are submitted to {quantum_backend.name}")
print(f"2. Monitor progress at: https://quantum.cloud.ibm.com/")
print(f"3. Use the check_job_status() function to monitor progress")
print(f"4. Jobs may take minutes to hours depending on queue length")
print(f"5. Run the results analysis code above when jobs complete")

print(f"\n Quick Status Check:")
print(f"quantum_job.status()          # Check first job")
print(f"quantum_job_mitigated.status() # Check second job")

print(f"\n Current Summary:")
print(f"  - Aer simulation:  Complete - {quantum_dist_aer}")
print(f"  - Hardware job (no mitigation):  {quantum_job.status()}")
print(f"  - Hardware job (with mitigation):  {quantum_job_mitigated.status()}")

C:\Users\user\AppData\Local\Temp\ipykernel_22996\2315228102.py:15: DeprecationWarning: The "ibm_quantum" channel option is deprecated and will be sunset on 1 July. After this date, "ibm_cloud" and "local" will be the only valid channels. For information on migrating to the new IBM Quantum Platform on the "ibm_cloud" channel, review the migration guide https://quantum.cloud.ibm.com/docs/migration-guides/classic-iqp-to-cloud-iqp .
  service = QiskitRuntimeService(channel="ibm_quantum")


✅ Service loaded from saved account

📡 Available backends:
  - ibm_brisbane: operational=True, queue=9
  - ibm_sherbrooke: operational=True, queue=17

🎯 Selected backend: ibm_brisbane
Backend status: <qiskit_ibm_runtime.models.backend_status.BackendStatus object at 0x000001B01BFCEA50>

🗺️  Plotting gate map for ibm_brisbane...
Could not plot gate map: "The 'Graphviz' library is required to use 'plot_coupling_map'.  To install, follow the instructions at https://graphviz.org/download/. Qiskit needs the Graphviz binaries, which the 'graphviz' package on pip does not install. You must install the actual Graphviz software."

🔔 Creating Bell circuit...
Bell circuit created:
        ┌───┐      ░ ┌─┐   
   q_0: ┤ H ├──■───░─┤M├───
        └───┘┌─┴─┐ ░ └╥┘┌─┐
   q_1: ─────┤ X ├─░──╫─┤M├
             └───┘ ░  ║ └╥┘
   c: 2/══════════════╬══╬═
                      ║  ║ 
meas: 2/══════════════╩══╩═
                      0  1 
Circuit diagram plotted

🔧 Transpiling circuit for ibm_brisbane with o

TypeError: SamplerV2.__init__() got an unexpected keyword argument 'backend'